In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
data = pd.read_csv("../input/severstal-steel-defect-detection/train.csv")
data.info()

#Here we prepare the dataset for a training 

# Here we get set of rows with notnull masks in the EncodedPixels column an set to 1

In [ ]:
defects = data[pd.notna(data.EncodedPixels)]
defects.EncodedPixels = 1
defects.info()
print(defects)

Here we get rows without data in the EncodedPixels column and set class to 0

In [ ]:
print((data.EncodedPixels).isnull())
NoDefects = data[(data.EncodedPixels).isnull()]
NoDefects.EncodedPixels = 0
NoDefects.info()
print(NoDefects)

Here we combine same number of images from both classes, and shuffle them

In [ ]:
dataset= NoDefects.sample(defects.shape[0])
dataset = dataset.append(defects,ignore_index=True)
dataset = dataset.sample(frac=1, replace=True, random_state=1)
dataset

show image example

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
filename = str(dataset.sample(1).ImageId_ClassId.values)[2:]
filename = filename[:-4]
filename = "../input/severstal-steel-defect-detection/train_images/"+filename
print(filename)

img=mpimg.imread(filename)
imgplot = plt.imshow(img)
plt.show()

In [ ]:
val = dataset[0:1000]
test = dataset[1000:2000]
train = dataset[2000:]
train.info

Convolutional network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=1e-5)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)


class Inception(nn.Module):

  def __init__(self):
    super(Inception, self).__init__()
    self.branch1x1 = BasicConv2d(128, 32, kernel_size=1, padding=0)
    self.branch1x1_2 = BasicConv2d(128, 32, kernel_size=1, padding=0)
    self.branch3x3_reduce = BasicConv2d(128, 24, kernel_size=1, padding=0)
    self.branch3x3 = BasicConv2d(24, 32, kernel_size=3, padding=1)
    self.branch3x3_reduce_2 = BasicConv2d(128, 24, kernel_size=1, padding=0)
    self.branch3x3_2 = BasicConv2d(24, 32, kernel_size=3, padding=1)
    self.branch3x3_3 = BasicConv2d(32, 32, kernel_size=3, padding=1)

  def forward(self, x):
    branch1x1 = self.branch1x1(x)

    branch1x1_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
    branch1x1_2 = self.branch1x1_2(branch1x1_pool)

    branch3x3_reduce = self.branch3x3_reduce(x)
    branch3x3 = self.branch3x3(branch3x3_reduce)

    branch3x3_reduce_2 = self.branch3x3_reduce_2(x)
    branch3x3_2 = self.branch3x3_2(branch3x3_reduce_2)
    branch3x3_3 = self.branch3x3_3(branch3x3_2)

    outputs = [branch1x1, branch1x1_2, branch3x3, branch3x3_3]
    return torch.cat(outputs, 1)


class CRelu(nn.Module):

  def __init__(self, in_channels, out_channels, **kwargs):
    super(CRelu, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
    self.bn = nn.BatchNorm2d(out_channels, eps=1e-5)

  def forward(self, x):
    x = self.conv(x)
    x = self.bn(x)
    x = torch.cat([x, -x], 1)
    x = F.relu(x, inplace=True)
    return x


class FaceBoxes(nn.Module):

  def __init__(self, phase, num_classes):
    super(FaceBoxes, self).__init__()
    self.phase = phase
    self.num_classes = num_classes

    self.conv1 = CRelu(3, 24, kernel_size=7, stride=4, padding=3)
    self.conv2 = CRelu(48, 64, kernel_size=5, stride=2, padding=2)

    self.inception1 = Inception()
    self.inception2 = Inception()
    self.inception3 = Inception()

    self.conv3_1 = BasicConv2d(128, 128, kernel_size=1, stride=1, padding=0)
    self.conv3_2 = BasicConv2d(128, 256, kernel_size=3, stride=2, padding=1)

    self.conv4_1 = BasicConv2d(256, 128, kernel_size=1, stride=1, padding=0)
    self.conv4_2 = BasicConv2d(128, 256, kernel_size=3, stride=2, padding=1)
    
    self.FC = nn.Linear(6656, 1, bias=True)

    self.softmax = nn.Softmax()
    
    self.sigmoid = nn.Sigmoid()

    if self.phase == 'train':
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m.bias is not None:
                    nn.init.xavier_normal_(m.weight.data)
                    m.bias.data.fill_(0.02)
                else:
                    m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

  
  def forward(self, x):

    x = self.conv1(x)
    x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
    x = self.conv2(x)
    x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
    x = self.inception1(x)
    x = self.inception2(x)
    x = self.inception3(x)

    x = self.conv3_1(x)
    x = self.conv3_2(x)

    x = self.conv4_1(x)
    x = self.conv4_2(x)
    x = x.reshape(-1)
    
    x = self.FC(x)
    
    output = self.sigmoid(x)
    
    return output

train network

In [ ]:
from sklearn.metrics import roc_auc_score
def validation():
    y_scores = [] # init array
    hog_features2 = []
    for i, filename in enumerate(test.ImageId_ClassId):
        filename = str(filename)
        filename = filename[:-2]
        filename = "../input/severstal-steel-defect-detection/train_images/" + filename
        img = mpimg.imread(filename)
        img = (img - 125)/256
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        img = img.transpose((2, 0, 1))
        out = net(Variable(torch.from_numpy(img).unsqueeze(0).float()).cuda())
        y_scores.append(out.item())
    y_true = test.EncodedPixels.values
    y_scores = np.array(y_scores)
    print ("ROC-AUC scores " + str(roc_auc_score(y_true, y_scores)))

In [ ]:
import cv2
from torch.autograd import Variable
import torch.optim as optim

num_classes = 2
momentum = 0.9
weight_decay = 5e-4
initial_lr = 1e-3


net = FaceBoxes('train', num_classes)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
optimizer = optim.Adam(net.parameters(), lr=initial_lr)
Crossloss = nn.CrossEntropyLoss()
error=[]
for epoch in range(1,25):
    print("epoch " +str(epoch))
    for i, filename in enumerate(train.ImageId_ClassId):
        filename = str(filename)
        filename = filename[:-2]
        filename = "../input/severstal-steel-defect-detection/train_images/" + filename
        img = mpimg.imread(filename)
        #img = cv2.resize(img, dsize=(600, 70), interpolation=cv2.INTER_CUBIC)

        img = (img - 125)/256
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        img = img.transpose((2, 0, 1))



        output = net(Variable(torch.from_numpy(img).unsqueeze(0).float()).cuda())
        target = torch.from_numpy(np.array(train.EncodedPixels.values[i]))
        optimizer.zero_grad()
        loss = - 0.001*(target * torch.log(output) + (1-target)* torch.log(1-output))
        loss.backward()
        optimizer.step()

        #print(str(i)+"/"+str(train.ImageId_ClassId.shape[0]) + " loss = " + str(1000 * loss.data[0]) + " target =" + str(target.data))
        error.append(1000*loss.item())
    print(" mean loss " + str(np.mean(np.array(error[-100:-1]))))
    validation()




learning process

In [ ]:
plt.plot(error)
plt.show()